In [1]:
try:
    import google.colab  # noqa: F401

    %pip install -q dataeval
except Exception:
    pass

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from torchvision.transforms.v2 import GaussianNoise

from dataeval.data import Embeddings, Metadata
from dataeval.detectors.drift import DriftCVM, DriftKS, DriftMMD
from dataeval.metrics.bias import label_parity
from dataeval.utils.datasets import VOCDetectionTorch

# Set a random seed
rng = np.random.default_rng(213)

# Set default torch device for notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
# Define the embedding network
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Load in pretrained resnet18 model
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # Add an additional fully connected layer with an embedding dimension of 128
        self.model.fc = nn.Linear(self.model.fc.in_features, 128)

    def forward(self, x):
        """Run input data through the model"""

        return self.model(x)

In [4]:
embedding_net = EmbeddingNet()

In [5]:
# Define pretrained model transformations
transforms = models.ResNet18_Weights.DEFAULT.transforms()

In [6]:
# Load the training dataset
train_ds = VOCDetectionTorch("./data", year="2011", image_set="train", download=False, transforms=transforms)
print(train_ds)

VOCDetectionTorch Dataset
-------------------------
    Year: 2011
    Transforms: [ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)]
    Image_set: train
    Metadata: {'id': 'VOCDetectionTorch_train', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'train'}
    Path: /dataeval/docs/source/notebooks/data/vocdataset/TrainVal/VOCdevkit/VOC2011
    Size: 5717


In [7]:
# Load the "operational" dataset
operational_ds = VOCDetectionTorch("./data", year="2011", image_set="val", download=False, transforms=transforms)
print(operational_ds)

VOCDetectionTorch Dataset
-------------------------
    Year: 2011
    Transforms: [ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)]
    Image_set: val
    Metadata: {'id': 'VOCDetectionTorch_val', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'val'}
    Path: /dataeval/docs/source/notebooks/data/vocdataset/TrainVal/VOCdevkit/VOC2011
    Size: 5823


In [8]:
# Create training batches and targets
train_embs = Embeddings(train_ds, batch_size=64, model=embedding_net, cache=True)

# Create operational batches and targets
operational_embs = Embeddings(operational_ds, batch_size=64, model=embedding_net, cache=True)

In [9]:
print(f"({len(train_embs)}, {train_embs[0].shape})")  # (5717, shape)
print(f"({len(operational_embs)}, {operational_embs[0].shape})")  # (5823, shape)

(5717, torch.Size([128]))
(5823, torch.Size([128]))


In [10]:
# A type alias for all of the drift detectors
DriftDetector = DriftMMD | DriftCVM | DriftKS

# Create a mapping for the detectors to iterate over
detectors: dict[str, DriftDetector] = {
    "MMD": DriftMMD(train_embs),
    "CVM": DriftCVM(train_embs),
    "KS": DriftKS(train_embs),
}

In [11]:
train_embs.to_tensor()

tensor([[-1.4559e-01, -1.1007e+00, -4.3497e-01,  ...,  1.5676e+00,
          3.6290e-02,  6.3820e-01],
        [-1.0709e+00, -1.4555e+00, -5.2617e-01,  ...,  9.0804e-01,
          6.9040e-01,  1.6711e+00],
        [ 7.6616e-02, -5.9417e-01, -1.1279e-01,  ...,  9.3638e-01,
          2.8131e-01,  1.5614e+00],
        ...,
        [-3.0425e-01, -1.0295e+00, -1.1580e+00,  ...,  6.7996e-01,
          6.6980e-01,  1.1827e+00],
        [-7.8196e-01, -6.6390e-01, -2.6067e-04,  ...,  5.1476e-01,
          8.0011e-01,  9.2813e-01],
        [-8.6152e-01, -8.9345e-01, -5.2310e-01,  ...,  6.2320e-01,
          7.9325e-01,  1.5175e+00]], device='cuda:0')

In [12]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(operational_embs).drifted}")

MMD detected drift? False
CVM detected drift? False


KS detected drift? False


In [13]:
# Applies gaussian noise to images before processing
noisy_embs = Embeddings(operational_ds, batch_size=64, model=embedding_net, transforms=GaussianNoise(), cache=True)

In [14]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(noisy_embs).drifted}")

MMD detected drift? True


CVM detected drift? True


KS detected drift? True


In [15]:
# Get the metadata for each dataset
train_md = Metadata(train_ds)
operational_md = Metadata(operational_ds)

# The VOC dataset has 20 classes
label_parity(train_md.class_labels, operational_md.class_labels, num_classes=20).p_value

np.float64(0.949856067521638)